<a href="https://colab.research.google.com/github/laulm/Angular/blob/master/Copia_de_Refactor_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook has the code to do my ETL process

ETL, which stands for extract, transform and load, is a data integration process that combines data from multiple data sources into a single, consistent data store that is loaded into a data warehouse or other target system.

# Content
* Step 1: Install libraries
* Step 2: Introduce the URL
* Step 3: Dowload the resource
* Step 4: Extract ZIP file
* Step 5: Load Data in Cloud Storage
* Step 6: Read CSV 
* Step 7: Visualize the Data 
* Step 8: Filter Data Using Like
* Step 9: Delete Columns 
* Step 10: Add a new column 
* Step 11: Save the CSV
* Step 12: Upload Data to Bigquery
* Step 13: Resources 


# Setup

## Install libraries

In [1]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import libraries

In [2]:
# built in libraries (Python)
import json
import os
import requests
from zipfile import ZipFile

# third-party libraries
from google.cloud import bigquery
from google.cloud import storage
from google.colab import drive
from google.oauth2 import service_account
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# your libraries


# Extraction

## Download dataset in CSV

### Introduce the URL

We introducing the URL, for calling this, later 

In [3]:
URL = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/geoelectorales/eceg_2020_csv.zip"

### Download the resource

1.   Import the request module and pandas to use these later
2.   Print a message indicating that the download has started
3.   Downloading the file by sending the request to the URL
4.   Split URL to get the file name
5.   Writing the file to the local file system
6.   Print a message to indicate the Downloading is completed





In [4]:
def download_raw_data(url: str ) -> any:
  """Download data from URL.
  
  This function downloads the data from a URL.

  Args:
    url (str): URL that contains free access the specific resource.
  
  Returns:
    True value (bool) if the download was OK, otherwise False value
  """

  print('Downloading started')

  # Downloading the file by sending the request to the URL
  req = requests.get(URL)
  
  # Split URL to get the file name
  filename = URL.split('/')[-1]
  print(filename)

  # Writing the file to the local file system
  with open(filename,'wb') as output_file:
    output_file.write(req.content)
    print('Downloading Completed')
    return True, filename
  
  return False, ""

In [5]:
result, filename = download_raw_data(url=URL)

eceg_2020_csv.zip


### Extract ZIP file

1. Import the zipfile 
2. With this, we extract all the content of zip file in current directory
3. Print a message to indicate the Extraction Completed

In [6]:
def unzip_file(path: str) -> any:
  """Unzip existing file.
  
  This function unzips the data from a ZIP file.

  Args:
    path (str): Resource path where the ZIP file is.
  
  Returns:
    True value (bool) if the download was OK, otherwise False value
  """
  with ZipFile(path, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    path_data = "./data"
    zipObj.extractall(path_data)
    print('Extraction Completed')
    return True, path_data

  return False, ""

In [7]:
if result == True:
  result_unzip, path_csv = unzip_file(path=filename)
  if result_unzip == True:
    print(f"The file was unzipped in {path_csv}!")    
else:
  print("The file was NOT unzipped!")    

Extraction Completed
The file was unzipped in ./data!


In [8]:
path_csv

'./data'

## Load raw (CSV) data in Cloud Storage

###Load Data in Cloud Storage

We go to load our data in google cloud storage
1. Import the "storage" and the "service_account" for using this more later 
2. Import os
3. Import json 

Give acces to the google cloud storage.

**Note**: To create your service account check this [Setting up authentication](https://cloud.google.com/storage/docs/reference/libraries#setting_up_authentication)

Don´t forget upload you service account .json

 Show the list of buckets from Google Storage

 **Note**: For more information about how to create a buckets check [Create Buckets](https://cloud.google.com/storage/docs/naming-buckets)

In [9]:
def get_client():
  return storage.Client.from_service_account_json( '/content/proyecto-prueba-353920-e3f554c0b04f.json') 

In [10]:
def validate_access() -> bool:
  """Validate bucket access.
  
  This function validates the access to a bucket in GCS.
  
  Returns:
    True value (bool) if the access was OK, otherwise False value
  """
  storage_client = get_client()
  buckets = storage_client.list_buckets()

  

  for bucket in buckets:
    if bucket.name: # if there is a bucket name
      print(bucket.name)
      return True
  
  return False

In [11]:
print(validate_access())

lflmexample
True


In [12]:
def get_client(type: str) -> any:
  """Return the type of client
  Args:
    type (str):Type of client
  Returns:
    any: This could be either a bigquery client or a cloud storage client
  """
  if type == "bigquery":
    print("Client BigQuery Created")
    return bigquery.Client.from_service_account_json( '/content/proyecto-prueba-353920-e3f554c0b04f.json')
  elif type == "cloud_storage":
    print("Client Cloud Storage Created")
    return storage.Client.from_service_account_json( '/content/proyecto-prueba-353920-e3f554c0b04f.json') 

  

In [13]:
bigquery_client=get_client(type="bigquery")
storage_client=get_client(type="cloud_storage")

Client BigQuery Created
Client Cloud Storage Created


In [14]:
def upload_file_gcs(client: any, bucket_name: str, file_name: str, path: str) -> bool:
  
  bucket = storage_client.get_bucket(bucket_name)
  path_ine_csv = file_name
  blob = bucket.blob(path_ine_csv)
  with open(path, "rb") as f:
    blob.upload_from_file(f)
  print("Upload completed")

In [15]:
path=path_csv + "/conjunto_de_datos/INE_DISTRITO_2020.CSV"
bucket_name="lflmexample"
file_name = os.path.basename(path)
upload_file_gcs(client=bigquery_client, bucket_name=bucket_name, file_name=file_name, path=path)

Upload completed


# Transformation

## Download Data from GCS

In [16]:
def download_file_gcs(client: any, bucket_name: str, file_name: str, path: str) -> bool:
  
  bucket = storage_client.get_bucket(bucket_name)
  path_ine_csv = file_name
  blob = bucket.blob(path_ine_csv)
  blob.download_to_filename(file_name)
  path_ine_csv


In [17]:
path=path_csv + "/conjunto_de_datos/INE_DISTRITO_2020.CSV"
bucket_name="lflmexample"
file_name = os.path.basename(path)
download_file_gcs(client=bigquery_client, bucket_name=bucket_name, file_name=file_name, path=path)

## Open Data with Pandas

#### Create a DataFrame

In [27]:
def create_df(data_frame: any):
  data_frame = pd.read_csv(path, index_col=0, encoding='latin-1') 
  return data_frame

In [ ]:
name_dataframe = file_name
create_df(data_frame=name_dataframe)


## Data Tranformations

### Filter Data

In [30]:
def filter_data(data_frame: any):
      data_filter = data_frame.filter(like='ration', axis=1)
      return data_filter

In [ ]:
data_f=create_df(data_frame=name_dataframe)
filter_data(data_frame=data_f)

### Delete Columns

In [33]:
#print("This is the Dataframe before deleting the column")
#print(file_name)
# Delete column with deth
def delete_column(data_frame: any):
    del data_frame["COMPLEJIDA"]
    print("This is the Dataframe after deleting column Duration")
    return data_frame

In [34]:
data_f=create_df(data_frame=name_dataframe)
delete_column(data_frame=data_f)

This is the Dataframe after deleting column Duration


,NOM_ENT,DISTRITO,INDIGENA,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_0A17,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCIN,VPH_SINTIC
ENTIDAD,,,,,,,,,,,,,,,,,,,,,
1,Aguascalientes,1,NO,482911,245249,237662,27689,13802,13887,174555,...,32683,110208,59381,45106,19305,14900,2752,8043,54801,985
1,Aguascalientes,2,NO,486085,249098,236987,24904,12319,12585,160970,...,51688,126004,84830,63907,33974,25751,1651,3813,40681,373
1,Aguascalientes,3,NO,456611,234577,222034,19271,9483,9788,127810,...,63447,123683,91792,65076,45445,29475,1618,3467,33514,353
2,Baja California,1,NO,485628,240417,245211,19179,9563,9616,139707,...,70615,141300,97720,72397,48338,29949,4220,5526,44317,1241
2,Baja California,2,NO,396169,197545,198624,11110,5479,5631,92281,...,82630,119147,97297,65908,49026,26747,2815,4302,26120,894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,Yucatán,5,SI,440998,221243,219755,22989,11341,11648,142542,...,9970,92325,36036,64935,5919,2653,10718,22202,73204,6014
32,Zacatecas,1,NO,401248,204384,196864,23355,11498,11857,139298,...,29959,89927,48632,51412,10315,8034,2495,10197,52570,1088
32,Zacatecas,2,NO,382209,195915,186294,20205,10044,10161,124394,...,34309,88813,44615,48299,7931,6273,4097,11035,57931,1678


### Add New Columns

In [35]:
#print("This is the Dataframe before add a column")
#file_name

def add_column(data_frame: any):
    a=np.random.randint(low=50, high= 101, size=len(data_frame))
    data_frame['Numeros_A']=a
    return data_frame

In [36]:
data_f=create_df(data_frame=name_dataframe)
add_column(data_frame=data_f)

,NOM_ENT,DISTRITO,INDIGENA,COMPLEJIDA,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCIN,VPH_SINTIC,Numeros_A
ENTIDAD,,,,,,,,,,,,,,,,,,,,,
1,Aguascalientes,1,NO,Disperso 1,482911,245249,237662,27689,13802,13887,...,110208,59381,45106,19305,14900,2752,8043,54801,985,74
1,Aguascalientes,2,NO,Concentrado 1,486085,249098,236987,24904,12319,12585,...,126004,84830,63907,33974,25751,1651,3813,40681,373,64
1,Aguascalientes,3,NO,Concentrado 2,456611,234577,222034,19271,9483,9788,...,123683,91792,65076,45445,29475,1618,3467,33514,353,84
2,Baja California,1,NO,Concentrado 2,485628,240417,245211,19179,9563,9616,...,141300,97720,72397,48338,29949,4220,5526,44317,1241,83
2,Baja California,2,NO,Concentrado 1,396169,197545,198624,11110,5479,5631,...,119147,97297,65908,49026,26747,2815,4302,26120,894,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,Yucatán,5,SI,Disperso 1,440998,221243,219755,22989,11341,11648,...,92325,36036,64935,5919,2653,10718,22202,73204,6014,52
32,Zacatecas,1,NO,Disperso 1,401248,204384,196864,23355,11498,11857,...,89927,48632,51412,10315,8034,2495,10197,52570,1088,57
32,Zacatecas,2,NO,Muy Disperso 1,382209,195915,186294,20205,10044,10161,...,88813,44615,48299,7931,6273,4097,11035,57931,1678,58


## Generate a new CSV with Pandas

# Loading

## Upload Data to Bigquery